In [5]:
import numpy as np

class DeepNeuralNetwork:
    def __init__(self, input_size=4, hidden1_size=5, hidden2_size=4, output_size=3, learning_rate=0.01):
        """Initialisation du réseau de neurones"""
        self.input_size = input_size
        self.hidden1_size = hidden1_size
        self.hidden2_size = hidden2_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.initialize_parameters()

    def initialize_parameters(self):
        """Initialisation des poids et biais"""
        # Poids et biais de la première couche cachée
        self.W1 = np.random.randn(self.input_size, self.hidden1_size) * 0.01
        self.b1 = np.zeros((1, self.hidden1_size))
        
        # Poids et biais de la deuxième couche cachée
        self.W2 = np.random.randn(self.hidden1_size, self.hidden2_size) * 0.01
        self.b2 = np.zeros((1, self.hidden2_size))
        
        # Poids et biais de la couche de sortie
        self.W3 = np.random.randn(self.hidden2_size, self.output_size) * 0.01
        self.b3 = np.zeros((1, self.output_size))

    def relu(self, Z):
        """Fonction d'activation ReLU"""
        return np.maximum(0, Z)

    def softmax(self, Z):
        """Fonction d'activation Softmax"""
        exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

    def forward_propagation(self, X):
        """Propagation avant"""
        # Première couche cachée
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.relu(self.Z1)
        
        # Deuxième couche cachée
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.relu(self.Z2)
        
        # Couche de sortie
        self.Z3 = np.dot(self.A2, self.W3) + self.b3
        self.A3 = self.softmax(self.Z3)
        
        return self.A3

    def compute_loss(self, y_true, y_pred):
        """Calcul de la perte (cross-entropy)"""
        return -np.mean(y_true * np.log(y_pred + 1e-8))

    def backward_propagation(self, X, y):
        """Rétropropagation"""
        m = X.shape[0]
        
        # Gradients de la couche de sortie
        dZ3 = self.A3 - y
        dW3 = np.dot(self.A2.T, dZ3) / m
        db3 = np.mean(dZ3, axis=0)
        
        # Gradients de la deuxième couche cachée
        dA2 = np.dot(dZ3, self.W3.T)
        dZ2 = dA2 * (self.Z2 > 0)  # Dérivée de ReLU
        dW2 = np.dot(self.A1.T, dZ2) / m
        db2 = np.mean(dZ2, axis=0)
        
        # Gradients de la première couche cachée
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * (self.Z1 > 0)  # Dérivée de ReLU
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.mean(dZ1, axis=0)
        
        # Mise à jour des poids et biais
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W3 -= self.learning_rate * dW3
        self.b3 -= self.learning_rate * db3

    def train(self, X, y, epochs=100):
        """Entraînement du réseau"""
        losses = []
        
        for epoch in range(epochs):
            # Propagation avant
            y_pred = self.forward_propagation(X)
            
            # Calcul de la perte
            loss = self.compute_loss(y, y_pred)
            losses.append(loss)
            
            # Rétropropagation
            self.backward_propagation(X, y)
            
            if epoch % 10 == 0:
                print(f"Époque {epoch}, Perte: {loss:.4f}")
        
        return losses

# Test du réseau
if __name__ == "__main__":
    # Création de données synthétiques pour le test
    np.random.seed(42)
    X = np.random.randn(100, 4)  # 100 échantillons avec 4 caractéristiques
    y = np.eye(3)[np.random.randint(0, 3, 100)]  # 3 classes

    # Création et entraînement du modèle
    model = DeepNeuralNetwork()
    print("Début de l'entraînement...")
    losses = model.train(X, y)

    # Test avec un nouvel échantillon
    test_sample = np.random.randn(1, 4)
    prediction = model.forward_propagation(test_sample)
    print("\nPrédiction pour un nouvel échantillon:")
    print(f"Probabilités des classes: {prediction[0]}")
    print(f"Classe prédite: {np.argmax(prediction)}")


Début de l'entraînement...
Époque 0, Perte: 0.3662
Époque 10, Perte: 0.3662
Époque 20, Perte: 0.3661
Époque 30, Perte: 0.3661
Époque 40, Perte: 0.3661
Époque 50, Perte: 0.3660
Époque 60, Perte: 0.3660
Époque 70, Perte: 0.3660
Époque 80, Perte: 0.3659
Époque 90, Perte: 0.3659

Prédiction pour un nouvel échantillon:
Probabilités des classes: [0.33234333 0.34093437 0.3267223 ]
Classe prédite: 1
